### 一. 依赖状态的类实现
#### 1. 什么叫依赖状态的类
* 如果类中的一些操作有基于状态的前提. 则这个类是依赖状态的
* 类库中包含许多依赖状态的同步类, 比如FutureTask, Semaphore, BlockingQueue等.  
  例如不能从一个控队类中删除元素, 或者不能获取一个尚未结束的任务的计算结果
  
#### 2. 多线程下状态依赖 - 等待条件变为真
* 单线程程序中, 条件分支依赖于某个状态, 如果这个条件不满足, 可以立刻失败, 因为该状态不会发生改变  
* 多线程程序中, 基于状态的条件可能会被其他线程修改, 所以不能立刻进入失败分支, 而是可以**等待条件变为真**

#### 3. 如何进行高效的条件等待
为了突出高效的条件等待方法 ,先来介绍一些低效的条件等待方法. 如下方法效率依次增高, 直到最后一种方法效率为最高:  
一个统一的场景, `有界队列实现`:   
&nbsp;&nbsp;&nbsp;&nbsp;有界队列的put和take往往包含一个前提条件,不能从空队列中获取元素, 也不能将元素放入已满的队列之中. 当条件不满足时, 可以阻塞直到对象进入正确的状态, 或抛出异常或返回一个错误状态. 如下是所有有界队列实现的父类, 定义了公用的`isFull()`和`isEmpty()`
```java
abstract class BaseBoundedQueue<V> {
    /**
     * 所有有界队列的父类, 子类来实现不同的状态依赖take(),put()方法
     */
    private final V[] buf;
    private int count;
    private int head;
    private int tail;

    BaseBoundedQueue(int capacity) {
        this.buf = (V[]) new Object[capacity];
    }

    protected synchronized final boolean isFull() {
        /** 判断队列是否空, 该方法因为和count状态有关, 因此需要对象的内置锁进行保护; 且该方法不允许子类覆盖*/
        return count == buf.length;
    }

    protected synchronized final boolean isEmpty() {
        return count == 0;
    }

    protected synchronized final void doPut(V v) {
        buf[tail++] = v;
        if (tail == buf.length)
            tail = 0;
        count++;
    }

    protected synchronized final V doTake() {
        V v = buf[head];
        buf[head++] = null;
        if (head == buf.length)
            head = 0;
        count--;
        return v;
    }

    protected abstract V take();

    protected abstract void put(V v);
}


```
1. 方法一 : **将条件检测的失败状态传递给调用者(抛异常或返回一个错误值)**   
   该方法下, 如果条件依赖检查失败, 直接在方法中抛异常通知调用者. 这看起来实现简单, 但使用起来有2个问题
    1. 依赖不满足只是当下不满足, 而不是发生了异常
    2. 调用者每次调用必须做好捕获异常的准备, 且在异常发生时主动重试
  ```java
    class GrumpyBoundedQueue<V> extends BaseBoundedQueue<V>{
        public GrumpyBoundedQueue(int capacity) {
            /** 构造器 */
            super(capacity);
        }

        @Override
        public synchronized V take() {
            /** 检查队列空, 则抛出异常 */
            if(isEmpty())
                throw new RuntimeException("empty queue");
            return doTake();
        }

        @Override
        public synchronized void put(V v) {
            /** 检查队列满, 抛出异常 */
            if(isFull())
                throw new RuntimeException("full queue");
            doPut(v);
        }
    }
  ```
    3. 客户调用者捕获异常后, 可以有两种重试选择 :   
        * 自行进入休眠   
        catch块中进行`Thread.sleep()`
        ```java
        while(true){
            try{
               V item = buffer.get();
               break;  // 未发生异常跳出循环
            }catch (Exception e){
               Thread.sleep(SLEEP_GRANULARITY)  // 自行休眠
            }
        }
        ```
        * 自旋等待(忙等待)  
        catch中调用`Thread.yield()`, 当调度器给该线程一个时间片时, yield选择让出处理器而不是消耗完整个CPU时间片
        ```java
        while(true){
            try{
               V item = buffer.get();
               break;  // 未发生异常跳出循环
            }catch (Exception e){
               Thread.yield();  // 自旋等待
            }
        }
        ```
* 方法二: **通过轮训和休眠的方式, 勉强的解决状态依赖问题**  
    1. 该方法也是先进行条件检查, 然后执行方法; 但会在检查失败后自行重试      
       首先, 状态条件的检查必须在锁的保护下进行.     
       其次, 如果测试失败, 当前执行的线程应该释放锁并休眠一段时间, 让其他线程能够访问内部的buf; 当线程醒来后, 将重新请求锁并再次进行重试. 因而, 线程将反复的在条件检查,休眠,获取锁释放锁的等待过程中切换.     
    2. 该方法实际将跑出异常那个方法中, 调用者的重试逻辑放到了方法内部进行; 使方法内部处理重试逻辑, 客户端只负责简单调用.
    ```java
    class SleepBoundedQueue<V> extends BaseBoundedQueue<V>{
        private static int SLEEP_TIME=1000;
        public SleepBoundedQueue(int capacity) {
            /** 构造器 */
            super(capacity);
        }

        @Override
        public V take() throws InterruptedException {
            /** 获取锁检查条件, 若队列空, 释放锁睡眠一段时间 */
            while(true){
                // 条件检查必须在持有锁的情况下进行
                synchronized (this){
                    if(!isEmpty())
                        return doTake();
                }
                Thread.sleep(SLEEP_TIME);
            }
        }

        @Override
        public void put(V v) throws InterruptedException {
            /** 获取锁检查条件, 若队列满, 释放锁睡眠一段时间 */
            while(true){
                synchronized (this){
                    if(!isFull())
                        doPut(v);
                }
                Thread.sleep(SLEEP_TIME);
            }
        }
    }
    ```
    3. 睡眠时间间隔选择   
       要选择合适的睡眠时间, 需要在响应性和CPU使用率之间权衡. 睡眠时间小, 响应性就高, 但消耗的CPU资源也高   
    4. 展望  
       这种通过轮训和睡眠实现阻塞的过程需要进行巨大努力. 如果存在某种挂起线程的方法, 能够保证当依赖条件为真时才立即醒来, 则将极大地提高效率, 这正是条件队列实现的功能.  
       
* 方法三: **使用条件队列**
    1. 条件队列:    
       条件队列的名字来源于: 使"等待某个条件为真"的线程加入到一个队列中.   
       一般队列中每个元素都是数据, 而条件队列中每个元素是一个线程
    2. 一个对象就是一个条件队列   
        * 正如每个对象都可以是一个锁一样, 每个对象也可以作为一个条件队列; 对象的wait(), notify()构成了使用内部条件队列的API   
        * 对象的内置锁和内置条件队列是相互关联的, 要调用对象X内置条件队列的任意一个方法, 必须先持有对象X的内置锁. 这也是因为 "等待由状态构成的条件" 和 "维护状态的一致性" 两种机制必须紧密地绑定在一起
        * wait():   
            1. 会释放锁, 并请求操作系统挂起当前线程. 使其他线程能够获取这个锁并修改对象的状态.   
            2. wait()醒来时, 将在返回之前重新获取锁.   
            3. wait()通常写在while循环内部. 由于代码可能出现bug, 线程不一定是在条件满足后醒来, 还可能在条件处于任意状态下被错误唤醒, 因此线程醒来后需要重新判断依赖条件是否为true  
        * 使用条件队列完成条件状态依赖, 和轮训休眠拥有相同的语义. 如果某个功能使用 "轮训和休眠" 无法完成, 那么使用条件队列也无法完成;    
        ```java
        class BoundedQueue<V> extends BaseBoundedQueue<V>{

            // 条件谓词: not-full (!isFull())
            // 条件谓词: not-empty (!isEmpty())

            public BoundedQueue(int capacity) {
                /** 构造器 */
                super(capacity);
            }

            @Override
            public synchronized V take() throws InterruptedException {
                /** 阻塞直到not-empty, this对象即是锁, 又是条件队列 */
                while(isEmpty()){
                    wait();
                }
                V v = doTake();
                notifyAll();
                return v;
            }

            @Override
            public synchronized void put(V v) throws InterruptedException {
                /** 阻塞直到not-full, this对象即是锁, 又是条件队列 */
                while(isFull()){
                    wait();
                }
                doPut(v);
                notifyAll();
            }
        }
        ```

### 二. 使用条件队列  

#### 1. 条件谓词


#### 2. 过早唤醒


#### 3. 丢失的信号

#### 4. 通知


#### 5. 示例: 条件队列实现阀门类

#### 6. 子类的安全问题



1. `boolean tryAcquire(int arg)`
2. `boolean tryRelease(int arg)`


1. `int tryAcquireShared(int arg)`
2. `boolean tryReleaseShared(int arg)`

### 二. 如何使用AbstractQueuedSynchronizer构建同步类
<img src="img/aqsdemo.png" width="70%">


### 三. 几个问题: 
#### 1. FutureTask如何实现阻塞版的get(), 等待任务完毕才能返回
1. java1.8中实现的FutureTask使用LockSupport实现任务的等待??  
    * (1) LockSupport中定义了线程阻塞,唤醒的方法; 定义了获取线程阻塞在哪个object上等方法
    * (2) `Unsafe`类的作用:
        1. 阻塞或唤醒线程 : 
            * `public native  void park(boolean isAbsolute,long time)`
            * `public native  void unpark(Object thread)`
        2. 获得内存地址上的值 : 
            * `public native double getDoubleVolatile(Object o,long offset)`
            * `public native float getFloat(long address)`
            * `public native float getFloat(Object o,long offset)`
        3. 修改内存地址的值
            * `public native  void putDouble(long address,double x)`
            * `public native  void putDouble(Object o,long offset,double x)`

     上面几个方法中的offset值内存的偏移地址, 此外Unsafe中定义了不同数据类型的单个偏移地址. 比如
     ```java
     ARRAY_LONG_INDEX_SCALE = theUnsafe.arrayIndexScale(long[].class);
     ```
     
2. FutureTask阻塞版get()的实现        
    1. FutureTask对任务的状态做了定义, 每个状态都是一个volatile的int值.    
       <img src="img/futuretask1.png" width="40%">
    2. 再来看一下FutureTask内部的一些属性:
        ```java
        /** 定义任务 */
        private Callable<V> callable;
        /** 任务执行的结果, 供get()发挥 */
        private Object outcome; // non-volatile, protected by state reads/writes
        /** 执行任务的线程 */
        private volatile Thread runner;
        /** 等待获取get的线程栈 */
        private volatile WaitNode waiters;
        ```
    2. FutureTask的run方法 :   
       在线程池中的本地线程执行Callable的call方法
       ```java
        public void run() {
            if (state != NEW ||
                // 1. CAS设置该FutureTask的runner对象为当前线程 runnerOffset=UNSAFE.objectFieldOffset(k.getDeclaredField("runner")
                !UNSAFE.compareAndSwapObject(this, runnerOffset, null, Thread.currentThread()))
                return;
            try {
                Callable<V> c = callable;
                if (c != null && state == NEW) {
                    V result;
                    boolean ran;
                    try {
                        result = c.call();  // 2. 执行任务
                        ran = true;
                    } catch (Throwable ex) {
                        result = null;
                        ran = false;
                        setException(ex);
                    }
                    if (ran)
                        set(result);
                }
            } finally {
                // runner must be non-null until state is settled to prevent concurrent calls to run()
                runner = null;
                // state must be re-read after nulling runner to prevent leaked interrupts
                int s = state;
                if (s >= INTERRUPTING)
                    handlePossibleCancellationInterrupt(s);
            }
        }
       ```
    2. get()方法首先判断当前任务的任务的状态:  
        * 如果还未运行完(<=Compiled), 则自旋等待任务运行完(awaitDone): 否则返回运行结果
            1. 在FutureTask结果的等待栈中加入一个新的节点WaitNode
            ```java
            static final class WaitNode {
                volatile Thread thread;
                volatile WaitNode next;
                WaitNode() { thread = Thread.currentThread(); }
            }
            ```
            2. 如果任务状态是COMPLETING, 表示还未运行完, 进入自旋等待(while(true){Thread.yield})
            ```java
            private int awaitDone(boolean timed, long nanos) throws InterruptedException {
                final long deadline = timed ? System.nanoTime() + nanos : 0L;  // 如果是有闲时间等待的话计算deadline
                WaitNode q = null;
                boolean queued = false;
                for (;;) {
                    if (Thread.interrupted()) {
                        removeWaiter(q);
                        throw new InterruptedException();
                    }

                    int s = state;
                    if (s > COMPLETING) {
                        if (q != null)
                            q.thread = null;
                        return s;
                    }
                    else if (s == COMPLETING) // 还未运行完, 自旋等待
                        Thread.yield();
                    else if (q == null)
                        q = new WaitNode();
                    else if (!queued)
                        //像等待栈中加入一个节点. waitersOffset = UNSAFE.objectFieldOffset(k.getDeclaredField("waiters"));
                        queued = UNSAFE.compareAndSwapObject(this, waitersOffset, q.next = waiters, q);
                    else if (timed) {
                        nanos = deadline - System.nanoTime();
                        if (nanos <= 0L) {
                            removeWaiter(q);
                            return state;
                        }
                        LockSupport.parkNanos(this, nanos);
                    }
                    else
                        LockSupport.park(this);
                    }
                }
            ```
    3.  cancel取消任务:
        因为runner属性记录了当前运行任务的线程, 所以调用运行任务线程的interupt()方法.所以, 只有在Callable的call()方法中设计了响应中断, 才能让cancel()方法生效. 
        ```java
        while(Thread.currentThread().isInterrupted() && ! flag)
        ```